In [ ]:
import pandas as pd
from great_tables import GT, loc, style

# Creating a table display from Paris 2024 Olympic Medals. 

## Preparing the data

Before creating the display table, we process the DataFrame.  We get Paris 2024 data, and we get the total, gold, silver and bronze medals by Committee and discipline.

In [ ]:
df_olympic = pd.read_csv("./input/olympic_medals.csv")
df_paris_2024 = df_olympic[df_olympic.Olympiad == "Paris 2024"]
df_paris_2024.head(2)

,Olympiad,Discipline,Event,Winner,Medal_type,Olympic_city,Olympic_year,Olympic_season,Gender,Code,Committee,Committee_type
20433,Paris 2024,Athletics,Women's 800m,Tsige Duguma,Silver,Paris,2024,summer,Women,ETH,Ethiopia,Country
20434,Paris 2024,Athletics,"Men's 10,000m",Berihu Aregawi,Silver,Paris,2024,summer,Men,ETH,Ethiopia,Country


In [ ]:
medal_counts = (
    df_paris_2024.groupby(["Committee", "Code", "Medal_type"])
    .size()
    .unstack(fill_value=0)  # Turn Medal_type into columns
    .reset_index()
)

medal_counts["Total"] = medal_counts[["Gold", "Silver", "Bronze"]].sum(axis=1)
medal_counts.sample(3)

Medal_type,Committee,Code,Bronze,Gold,Silver,Total
10,Brazil,BRA,20,6,14,40
61,Mongolia,MGL,0,0,2,2
83,South South Korea,KOR,10,13,9,32


In [ ]:
# Sort by Discipline, then by Gold → Silver → Bronze (descending)
medal_counts = medal_counts.sort_values(
    by=["Gold", "Silver", "Bronze"], ascending=[False, False, False]
)
medal_counts.head(3)

Medal_type,Committee,Code,Bronze,Gold,Silver,Total
93,United States,USA,84,80,88,252
15,China,CHN,48,80,54,182
51,Japan,JPN,26,40,24,90


In [5]:
# Group also by Gender to get medal counts per gender
gender_counts = (
    df_paris_2024[df_paris_2024["Medal_type"].isin(["Gold", "Silver", "Bronze"])]
    .groupby(["Committee", "Gender", "Medal_type"])
    .size()
    .unstack("Medal_type", fill_value=0)
    .reindex(columns=["Gold", "Silver", "Bronze"], fill_value=0)
)

In [ ]:
gender_flat = gender_counts.stack(level=0).unstack(  # Stack Gender
    "Gender"
)  # Gender to columns

In [7]:
gender_flat

Gender                 Men  Mixed  Open  Openpen  Women
Committee  Medal_type                                  
Albania    Gold        0.0    NaN   NaN      NaN    NaN
           Silver      0.0    NaN   NaN      NaN    NaN
           Bronze      4.0    NaN   NaN      NaN    NaN
Algeria    Gold        0.0    NaN   NaN      NaN    4.0
           Silver      0.0    NaN   NaN      NaN    0.0
...                    ...    ...   ...      ...    ...
Uzbekistan Silver      2.0    NaN   NaN      NaN    2.0
           Bronze      6.0    NaN   NaN      NaN    0.0
Zambia     Gold        0.0    NaN   NaN      NaN    NaN
           Silver      0.0    NaN   NaN      NaN    NaN
           Bronze      2.0    NaN   NaN      NaN    NaN

[288 rows x 5 columns]

In [ ]:
gender_flat = gender_counts.unstack(
    "Gender", fill_value=0
)  # Gender becomes outer column level

In [9]:
gender_flat.columns = [f"{medal}_{gender}" for medal, gender in gender_flat.columns]
gender_flat.reset_index(inplace=True)

In [10]:
gender_flat = gender_flat.reset_index(drop=True)
# Merge with your existing medal_counts
medal_counts = medal_counts.merge(gender_flat, on=["Committee"], how="left")

In [11]:
df_flags = pd.read_csv("./input/flags.csv")
df_flags.head(2)

,Committee,Code,Flag
0,Albania,ALB,🇦🇱
1,Algeria,ALG,🇩🇿


In [12]:
medal_counts = medal_counts.merge(df_flags[["Code", "Flag"]], on="Code")

In [ ]:
medal_counts["Committee"] = medal_counts["Flag"] + " " + medal_counts["Committee"]

In [14]:
medal_counts = medal_counts.drop(columns=["Code", "Flag"])

In [ ]:
proper_order = [
    "Committee",
    "Total",
    "Gold",
    "Silver",
    "Bronze",
    "Gold_Men",
    "Silver_Men",
    "Bronze_Men",
    "Gold_Women",
    "Silver_Women",
    "Bronze_Women",
    "Gold_Mixed",
    "Silver_Mixed",
    "Bronze_Mixed",
    "Gold_Open",
    "Silver_Open",
    "Bronze_Open",
]

# Reorder the dataframe
medal_counts = medal_counts[proper_order]

In [ ]:
table = (
    GT(medal_counts.head(10), rowname_col="Committee")
    .tab_header(
        title="Paris 2024 Medals - Top 10",
        subtitle="Medals by Committee | Total and by Gender",
    )
    .tab_spanner(label="Total", columns=["Total", "Gold", "Silver", "Bronze"])
    .tab_spanner(label="Men", columns=["Gold_Men", "Silver_Men", "Bronze_Men"])
    .tab_spanner(label="Women", columns=["Gold_Women", "Silver_Women", "Bronze_Women"])
    .tab_spanner(label="Mixed", columns=["Gold_Mixed", "Silver_Mixed", "Bronze_Mixed"])
    .tab_spanner(label="Open", columns=["Gold_Open", "Silver_Open", "Bronze_Open"])
    .cols_label(
        Total="All",
        Gold="🥇",
        Silver="🥈",
        Bronze="🥉",
        Gold_Men="🥇",
        Silver_Men="🥈",
        Bronze_Men="🥉",
        Gold_Women="🥇",
        Silver_Women="🥈",
        Bronze_Women="🥉",
        Gold_Mixed="🥇",
        Silver_Mixed="🥈",
        Bronze_Mixed="🥉",
        Gold_Open="🥇",
        Silver_Open="🥈",
        Bronze_Open="🥉",
    )
    .tab_style(
        style=style.fill(color="#C6BCBC"),
        locations=loc.body(columns=["Total"]),
    )
    .tab_style(
        style=style.fill(color="#fefefe"),
        locations=loc.body(columns=["Gold", "Silver", "Bronze"]),
    )
    .tab_style(
        style=style.fill(color="#dbd9e9"),
        locations=loc.body(columns=["Gold_Men", "Silver_Men", "Bronze_Men"]),
    )
    .tab_style(
        style=style.fill(color="#ffb6b6"),
        locations=loc.body(columns=["Gold_Women", "Silver_Women", "Bronze_Women"]),
    )
    .tab_style(
        style=style.fill(color="#fefefe"),
        locations=loc.body(columns=["Gold_Mixed", "Silver_Mixed", "Bronze_Mixed"]),
    )
    .tab_style(
        style=style.fill(color="#efefef"),
        locations=loc.body(columns=["Gold_Open", "Silver_Open", "Bronze_Open"]),
    )
)

table

GT(_tbl_data=          Committee  Total  Gold  Silver  Bronze  Gold_Men  Silver_Men  \
0  🇺🇸 United States    252    80      88      84        26          32   
1          🇨🇳 China    182    80      54      48        34          20   
2          🇯🇵 Japan     90    40      24      26        24          14   
3      🇦🇺 Australia    106    36      38      32        10          18   
4         🇫🇷 France    128    32      52      44        22          28   
5    🇳🇱 Netherlands     68    30      14      24        12           4   
6  🇬🇧 Great Britain    130    28      44      58        12          30   
7          🇮🇹 Italy     80    24      26      30         6          18   
8        🇩🇪 Germany     66    24      26      16         8          10   
9    🇳🇿 New Zealand     40    20      14       6         4           6   

   Bronze_Men  Gold_Women  Silver_Women  Bronze_Women  Gold_Mixed  \
0          46          52            46            36           2   
1          14          38            30            32           6   
2           8          16             6            14           0   
3          12          26            18            16           0   
4          26           8            22            16           2   
5           6          16            10            16           2   
6          18          12            14            30           0   
7          22          14             8             8           4   
8           4           6            12            12           2   
9           0          16             8             4           0   

   Silver_Mixed  Bronze_Mixed  Gold_Open  Silver_Open  Bronze_Open  
0             6             2          0            2            0  
1             4             2          1            0            0  
2             4             2          0            0            1  
3             0             4          0            1            0  
4             0             0          0            1            1  
5             0             0          0            0            1  
6             0             4          2            0            3  
7             0             0          0            0            0  
8             2             0          4            1            0  
9             0             2          0            0            0  , _body=<great_tables._gt_data.Body object at 0x747bc84adb80>, _boxhead=Boxhead([ColInfo(var='Committee', type=<ColInfoTypeEnum.stub: 2>, column_label='Committee', column_align='left', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='All', column_align='right', column_width=None), ColInfo(var='Gold', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(va

In [17]:
table.save("./output/country_medals.png", scale=4)

GT(_tbl_data=          Committee  Total  Gold  Silver  Bronze  Gold_Men  Silver_Men  \
0  🇺🇸 United States    252    80      88      84        26          32   
1          🇨🇳 China    182    80      54      48        34          20   
2          🇯🇵 Japan     90    40      24      26        24          14   
3      🇦🇺 Australia    106    36      38      32        10          18   
4         🇫🇷 France    128    32      52      44        22          28   
5    🇳🇱 Netherlands     68    30      14      24        12           4   
6  🇬🇧 Great Britain    130    28      44      58        12          30   
7          🇮🇹 Italy     80    24      26      30         6          18   
8        🇩🇪 Germany     66    24      26      16         8          10   
9    🇳🇿 New Zealand     40    20      14       6         4           6   

   Bronze_Men  Gold_Women  Silver_Women  Bronze_Women  Gold_Mixed  \
0          46          52            46            36           2   
1          14          38            30            32           6   
2           8          16             6            14           0   
3          12          26            18            16           0   
4          26           8            22            16           2   
5           6          16            10            16           2   
6          18          12            14            30           0   
7          22          14             8             8           4   
8           4           6            12            12           2   
9           0          16             8             4           0   

   Silver_Mixed  Bronze_Mixed  Gold_Open  Silver_Open  Bronze_Open  
0             6             2          0            2            0  
1             4             2          1            0            0  
2             4             2          0            0            1  
3             0             4          0            1            0  
4             0             0          0            1            1  
5             0             0          0            0            1  
6             0             4          2            0            3  
7             0             0          0            0            0  
8             2             0          4            1            0  
9             0             2          0            0            0  , _body=<great_tables._gt_data.Body object at 0x747bc84adb80>, _boxhead=Boxhead([ColInfo(var='Committee', type=<ColInfoTypeEnum.stub: 2>, column_label='Committee', column_align='left', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='All', column_align='right', column_width=None), ColInfo(var='Gold', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Men', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(var='Bronze_Women', type=<ColInfoTypeEnum.default: 1>, column_label='🥉', column_align='right', column_width=None), ColInfo(var='Gold_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥇', column_align='right', column_width=None), ColInfo(var='Silver_Mixed', type=<ColInfoTypeEnum.default: 1>, column_label='🥈', column_align='right', column_width=None), ColInfo(va

In [18]:
table.write_raw_html("./output/country_medals.html")

## Charts

In [23]:
top_ten = medal_counts.head(10)

In [28]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('cairo')

plt.bar(
    top_ten["Committee"],
    top_ten["Total"],
    color="skyblue",
    edgecolor="black",
)
plt.xticks(rotation=45, ha="right")
plt.xlabel("Committee")
plt.ylabel("Total 🥇")
plt.title("Total Medals by Committee 🇯🇵🇨🇦🇺🇸")
plt.show()

ImportError: cairo backend requires that pycairo>=1.14.0 or cairocffi is installed